In [325]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import numpy as np

In [326]:
PROBLEM_SIZE = 5
NUM_SETS = 10
SET_PROBABILITY = 0.3
SETS = tuple(
    np.array([random() <SET_PROBABILITY for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken', 'not_taken'])

In [327]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

def distance(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))

def count_taken_sets(state):
    return len(state.taken)



In [100]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"

In [328]:
# frontier = PriorityQueue()
frontier = SimpleQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((distance(new_state), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

This is a function to implement a simple search without priority for set covering, allowing to specify just the data structure on which to memorize the frontier.
A SimpleQueue corresponds to a Breadth-first search.
A LifoQueue corresponds to a Depth-first search.

In [329]:
def set_covering_basic_search(current_state, frontier=SimpleQueue()):
    
    frontier.put(current_state)
    counter = 0
    current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put(new_state)
        current_state = frontier.get()
    
    print(
        f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
    )
    print(f"Solution: {current_state}")

Breadth-first search
Solved in 13 steps (2 tiles)
Solution: State(taken={1, 2}, not_taken={0, 3, 4, 5})
Depth-first search
Solved in 3 steps (3 tiles)
Solution: State(taken={3, 4, 5}, not_taken={0, 1, 2})
Greedy best-first search
Solved in 2 steps (2 tiles)
Solution: State(taken={1, 2}, not_taken={0, 3, 4, 5})
A* search
Solved in 3 steps (2 tiles)
Solution: State(taken={1, 2}, not_taken={0, 3, 4, 5})


In [323]:
current_state = State(set(), set(range(NUM_SETS)))
print("Breadth-first search using SimpleQueue")
set_covering_basic_search(current_state)
print("Depth-first search using LifoQueue")
set_covering_basic_search(current_state, LifoQueue())

A* search
Solved in 4 steps (2 tiles)
Solution: State(taken={3, 4}, not_taken={0, 1, 2, 5})


In [259]:
SETS

This is a priority search function that allows to specify the priority function passed to establish the order of the element memorized in a priority queue
If the function is not specified , it is used a simple function that compute the distance from the actual state to the goal state, resulting in a greedy best first search strategy

In [259]:
def set_covering_priority_search(current_state, priority_func=distance):
    
    frontier = PriorityQueue()
    state = State(set(), set(range(NUM_SETS)))
    frontier.put((priority_func(state), state))
    
    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((priority_func(new_state), new_state))
        _, current_state = frontier.get()
    
    print(
        f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
    )
    print(f"Solution: {current_state}")

In [ ]:
current_state = State(set(), set(range(NUM_SETS)))
print("Greedy Best-first search")
set_covering_priority_search(current_state)
print("custom example")
set_covering_priority_search(current_state, priority_func=count_taken_sets)


In [84]:
def set_covering_search(current_state, frontier=None, priority_func= None):
    
    if frontier is None:
        frontier = PriorityQueue()
    if priority_func is None:
        priority_func = lambda _: None
    state = State(set(), set(range(NUM_SETS)))
    frontier.put((priority_func(state), state))
    
    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((priority_func(new_state), new_state))
        _, current_state = frontier.get()
    
    print(
        f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
    )
    print(f"Solution: {current_state}")

State(taken=set(), not_taken={0, 1, 2, 3, 4})
Breadth-first search 1
Solved in 4 steps (3 tiles)
Solution: State(taken={0, 1, 4}, not_taken={2, 3})
Depth-first search using LifoQueue 1
Solved in 4 steps (4 tiles)
Solution: State(taken={1, 2, 3, 4}, not_taken={0})
Greedy Best-first search 1
Solved in 2 steps (2 tiles)
Solution: State(taken={1, 4}, not_taken={0, 2, 3})
Breadth-first search 2
Solved in 13 steps (2 tiles)
Solution: State(taken={1, 4}, not_taken={0, 2, 3})
Depth-first search using LifoQueue 2
Solved in 4 steps (4 tiles)
Solution: State(taken={1, 2, 3, 4}, not_taken={0})
Greedy Best-first search 2
Solved in 2 steps (2 tiles)
Solution: State(taken={1, 4}, not_taken={0, 2, 3})
